# Import

In [1]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml


# URL Rule

In [ ]:
# comp_id = 380725    # 기업 고유 UID
p_num = 51          # 리뷰 페이지 (4 reviews / page)

### query로 CompID 가져오기

In [ ]:
Keyword = '삼성전자'

In [ ]:
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(Keyword))
print(search_req.text)

In [ ]:
search_soup = BeautifulSoup(search_req.text, 'lxml')
search_soup.text

In [ ]:
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

In [ ]:
comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
comp_id

In [ ]:
URL = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=4&isNew=false&employType=1&isEmploy=true&jobCode='
print(URL)

# Json to DataFrame 

In [ ]:
# Json 형식의 웹 데이터 가져오기
raw_data = requests.get(URL).json()
print(raw_data)

In [ ]:
data = raw_data['reviewList']
data

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df.columns

In [ ]:
df.drop(['CI', 'Gender2', 'Answer', 'UsefulY', 'CareerYearYN',  'MyUsefulY', 'MyOpinion', 'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
# 정규식으로 이스케이프 문자 제거        # Good, Bad 리뷰 항목에만 적용.
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df

In [ ]:
# RegDate 컬럼의 데이터 - 문자열 9번째 까지 출력
df['RegDate'] = df['RegDate'].apply(lambda x: x[:10])
df

# For 문으로 만들어보기
    - 리뷰 데이터가 있는 페이지까지 반복

In [ ]:
rv = []

for p_num in range(1, 52) :
    url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=4&isNew=false&employType=1&isEmploy=true&jobCode='
    rv.append(requests.get(url).json()['reviewList'])


data = list(itertools.chain.from_iterable(rv))
df = pd.DataFrame(data)


df.drop(['CI', 'Gender2', 'Answer', 'UsefulY', 'CareerYearYN',  'MyUsefulY', 'MyOpinion', 'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], axis=1, inplace=True)
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())


df

# While 문으로 만들기 
    - 그 다음 페이지에 리뷰 데이터 개수가 0 이면 멈춤


In [ ]:
# len(requests.get(url).json()['reviewList'])

In [26]:
comp_name = '삼성전자'
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(comp_name)) # Keyword = comp_name
# print(search_req.text)

search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
comp_id

'380725'

In [27]:

try : 
    # comp_id = 380725
    p_num = 1

    rv = []


    while True : 

        url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=2000&isNew=false&employType=1&isEmploy=false&jobCode='
        rv.append(requests.get(url).json()['reviewList'])

        p_num += 1

        if len(requests.get(url).json()['reviewList']) == 0 :
            break


    data = list(itertools.chain.from_iterable(rv))
    df = pd.DataFrame(data)

    df.drop(['CI', 'Gender2', 'Answer', 'UsefulY', 'CareerYearYN',  'MyUsefulY', 'MyOpinion', 
            'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
            axis=1, inplace=True)
    df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())
    


except :
    print("리뷰가 존재하지 않습니다.")


df


,idx,CompID,CompName,RegDate,EmployText,RecomName,EmployType,NewOld,Good,Bad,CareerYear,JobName,MyStarScore
0,209151,380725,삼성전자,20230810,현직,추천,정규직,신입,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,13,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,전직,추천,정규직,신입,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,3,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,전직,추천,정규직,신입,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,3,웹개발,4.6
3,208095,380725,삼성전자,20230626,현직,추천,정규직,신입,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,1,일반영업,2.6
4,207978,380725,삼성전자,20230621,전직,비추,정규직,경력,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,14,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,전직,추천,정규직,경력,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,4,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,현직,추천,정규직,신입,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,13,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,전직,추천,정규직,신입,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,3,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,전직,추천,정규직,경력,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",11,네트워크/서버/보안,3.4


# 컬럼명 및 데이터 변경

### 컬럼명 변경

In [28]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'RecomName',
       'EmployType', 'NewOld', 'Good', 'Bad', 'CareerYear', 'JobName',
       'MyStarScore'],
      dtype='object')

In [29]:
df.rename(columns=
          {'CompName':'comp_name', 
           'RegDate':'review_date', 
           'EmployText':'is_office', 
           'Good':'review_pos', 
           'Bad':'review_neg', 
           'MyStarScore':'review_rate', 
           'JobName':'position'
           }, inplace=True)

df

,idx,CompID,comp_name,review_date,is_office,RecomName,EmployType,NewOld,review_pos,review_neg,CareerYear,position,review_rate
0,209151,380725,삼성전자,20230810,현직,추천,정규직,신입,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,13,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,전직,추천,정규직,신입,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,3,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,전직,추천,정규직,신입,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,3,웹개발,4.6
3,208095,380725,삼성전자,20230626,현직,추천,정규직,신입,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,1,일반영업,2.6
4,207978,380725,삼성전자,20230621,전직,비추,정규직,경력,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,14,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,전직,추천,정규직,경력,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,4,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,현직,추천,정규직,신입,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,13,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,전직,추천,정규직,신입,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,3,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,전직,추천,정규직,경력,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",11,네트워크/서버/보안,3.4


In [30]:
df.columns

Index(['idx', 'CompID', 'comp_name', 'review_date', 'is_office', 'RecomName',
       'EmployType', 'NewOld', 'review_pos', 'review_neg', 'CareerYear',
       'position', 'review_rate'],
      dtype='object')

### 전/현직 여부를 Boolean 값으로 변경

In [114]:
df['is_office']

0      현직
1      전직
2      전직
3      현직
4      전직
       ..
301    전직
302    현직
303    전직
304    전직
305    전직
Name: is_office, Length: 306, dtype: object

In [115]:
df['is_office'] = df['is_office'].apply(lambda x: x.replace('현직', '1').strip() == '1')

In [116]:
df['is_office']

0       True
1      False
2      False
3       True
4      False
       ...  
301    False
302     True
303    False
304    False
305    False
Name: is_office, Length: 306, dtype: bool

# DataFrame to CSV file

In [ ]:
SAVE_PATH = r'./data/'

In [ ]:
df['CompName'][0]

In [ ]:
comp = df['CompName'][0]

In [ ]:
os.makedirs(SAVE_PATH, exist_ok=True)

# csv 파일로 저장.

file_name = f"{comp}_reviews.csv"
save_file_path = os.path.join(SAVE_PATH, file_name)

df.to_csv(save_file_path, index=False, encoding = "utf-8")

# 함수화

In [2]:
SAVE_PATH = r'./data/'

In [33]:
def RV_Catch(comp = str): 
    
    search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
    search_req = requests.get(search_url.format(comp)) # Keyword = comp_name
    # print(search_req.text)

    search_soup = BeautifulSoup(search_req.text, 'lxml')
    # search_soup.text
    search_soup.select('li:nth-child(1) > div.txt > p.name > a')

    comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))





    p_num = 1

    rv = []



    try : 

        while True : 

            url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=2000&isNew=false&employType=1&isEmploy=false&jobCode='
            rv.append(requests.get(url).json()['reviewList'])

            p_num += 1

            if len(requests.get(url).json()['reviewList']) == 0 :
                break


        data = list(itertools.chain.from_iterable(rv))
        df = pd.DataFrame(data)


        df.drop(['CI', 'Gender2', 'Answer', 'UsefulY', 'CareerYearYN',  'MyUsefulY', 'MyOpinion', 
                'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
                axis=1, inplace=True)
        
        df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
        df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())

        df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())

        df['EmployText'] = df['EmployText'].apply(lambda x: x.replace('현직', '1').strip() == '1')



        

        
        
        
        # csv 파일로 저장.

        comp = df['CompName'][0]

        os.makedirs(SAVE_PATH, exist_ok=True)

        file_name = f"{comp}_reviews.csv"
        save_file_path = os.path.join(SAVE_PATH, file_name)

        df.to_csv(save_file_path, index=False, encoding = "utf-8")
        


        
    except : 
        print("리뷰가 존재하지 않습니다.")




In [34]:
RV_Catch('SK하이닉')

In [19]:
RV_Catch('화인석재')

리뷰가 존재하지 않습니다.


In [18]:
RV_Catch('삼성전자')

In [35]:
RV_Catch('네이버')

In [122]:
RV_Catch('카카오')